<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/gliner/gliner2_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet --upgrade gliner2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.9/151.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 996.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.7 MB/s eta 0:00:00


In [2]:
from gliner2 import GLiNER2

# Load model once, use everywhere
extractor = GLiNER2.from_pretrained("fastino/gliner2-large-v1")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

🧠 Model Configuration
Encoder model      : microsoft/deberta-v3-large
Counting layer     : count_lstm
Token pooling      : first


model.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

In [3]:
text = """Augusta Ada King, Countess of Lovelace (10 December 1815 – 27 November 1852), also known as Ada Lovelace, was an English mathematician and writer chiefly known for work on Charles Babbage's proposed mechanical general-purpose computer, the analytical engine. She was the first to recognise the machine had applications beyond pure calculation. Lovelace is often considered the first computer programmer.

Lovelace was the only legitimate child of poet Lord Byron and reformer Anne Isabella Milbanke. All her half-siblings, Lord Byron's other children, were born out of wedlock to other women. Lord Byron separated from his wife a month after Ada was born, and left England forever. He died in Greece during the Greek War of Independence, when she was eight. Lady Byron was anxious about her daughter's upbringing and promoted Lovelace's interest in mathematics and logic, to prevent her developing her father's perceived insanity. Despite this, Lovelace remained interested in her father, naming one son Byron and the other, for her father's middle name, Gordon. Lovelace was buried next to her father at her request.

Although often ill in childhood, Lovelace pursued her studies assiduously. She married William King in 1835. King was a Baron, and was created Viscount Ockham and 1st Earl of Lovelace in 1838. The name Lovelace was chosen because Ada was descended from the extinct Baron Lovelaces.[4] The title given to her husband thus made Ada the Countess of Lovelace."""

In [4]:
entities = extractor.extract_entities(
    text,
    {
      "Person": "Names of people.",
      "Country": "Countries and geographic locations.",
      "Work of art": "Titles of books, inventions, or specific machines.",
      "Event": "Historical events or conflicts.",
      "Product": "Technological objects or mechanical systems."
    }
)
entities

{'entities': {'Person': ['Ada Lovelace',
   'Anne Isabella Milbanke',
   'William King',
   'Lord Byron',
   'Charles Babbage',
   'Augusta Ada King',
   'Lady Byron',
   'Byron',
   'Viscount Ockham',
   'Gordon'],
  'Country': ['Greece', 'England'],
  'Work of art': ['analytical engine'],
  'Event': ['Greek War of Independence'],
  'Product': ['analytical engine']}}

In [17]:
schema = extractor.create_schema().relations({
    "works_for": "Employment relationship where person works at organization",
    "founded": "Founding relationship where person created organization",
    "acquired": "Acquisition relationship where company bought another company",
    "located_in": "Geographic relationship where entity is in a location",
    "parent": "Parent of a person"
})

results = extractor.extract(text, schema, include_confidence=True)
results

{'relation_extraction': {'works_for': [{'head': {'text': 'Augusta Ada King',
     'confidence': 0.7108697891235352},
    'tail': {'text': 'analytical engine', 'confidence': 0.7266157865524292}}],
  'founded': [{'head': {'text': 'Charles Babbage',
     'confidence': 0.9872558116912842},
    'tail': {'text': 'analytical engine', 'confidence': 0.9464283585548401}}],
  'acquired': [],
  'located_in': [{'head': {'text': 'Lord Byron',
     'confidence': 0.8456583619117737},
    'tail': {'text': 'Greece', 'confidence': 0.9048344492912292}}],
  'parent': [{'head': {'text': 'Lord Byron', 'confidence': 0.923921525478363},
    'tail': {'text': 'Ada Lovelace', 'confidence': 0.988991379737854}},
   {'head': {'text': 'Anne Isabella Milbanke',
     'confidence': 0.9757091999053955},
    'tail': {'text': 'Ada Lovelace', 'confidence': 0.9925393462181091}}]}}

In [14]:
schema = (extractor.create_schema()
    .entities(["person", "organization", "location"])
    .relations(["works_for", "located_in"])
)
results = extractor.extract(text, schema, include_confidence=True)
results

{'entities': {'person': [{'text': 'Lord Byron',
    'confidence': 0.9957078695297241},
   {'text': 'Anne Isabella Milbanke', 'confidence': 0.9951424598693848},
   {'text': 'William King', 'confidence': 0.99454665184021},
   {'text': 'Ada Lovelace', 'confidence': 0.9936741590499878},
   {'text': 'Charles Babbage', 'confidence': 0.9884392619132996},
   {'text': 'Augusta Ada King', 'confidence': 0.9880245327949524},
   {'text': 'Lady Byron', 'confidence': 0.8894632458686829},
   {'text': 'Byron', 'confidence': 0.860332190990448},
   {'text': 'Gordon', 'confidence': 0.6732606291770935}],
  'organization': [{'text': 'analytical engine',
    'confidence': 0.8495214581489563}],
  'location': [{'text': 'Greece', 'confidence': 0.9944837689399719},
   {'text': 'England', 'confidence': 0.9782404899597168}]},
 'relation_extraction': {'works_for': [{'head': {'text': 'Augusta Ada King',
     'confidence': 0.8782658576965332},
    'tail': {'text': 'analytical engine', 'confidence': 0.7176190614700317

In [15]:
result = extractor.extract_json(
    text,
    {
        "Person": [
            "name::str::Person name",
            "summary::str::Summary of the person information"
        ]
    }
)
result

{'Person': [{'name': 'Augusta Ada King', 'summary': None}]}

In [16]:
result = extractor.extract_json(
    text.split("\n\n")[0],
    {
        "Person": [
            "name::str::name of person",
        ]
    }
)
result

{'Person': [{'name': 'Augusta Ada King'}]}